In [1]:
displayHTML("<font size=8>Let's start building simple ELEMENTS of a <font size=8 color='green'>PIPELINE</font> for</font> <font color=orange size=8>Orange Churn dataset</font>")

![How to create a DataFrame](https://blog.cloudera.com/wp-content/uploads/2017/04/Spark.png)

### [MSTC](http://mstc.ssr.upm.es/big-data-track) and MUIT:

## Importing Churn Data

###  Load churn-bigml-80.csv into a DataFrame

In [5]:
%fs ls /FileStore/tables

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

CV_data = sqlContext.read.load('/FileStore/tables/churn_bigml_80-bf1a8.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')


In [7]:
display(CV_data)

In [8]:
CV_data.printSchema()

This is simply to illustrate an example to apply a UDF to a Spark DataFrame

In [10]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.functions import UserDefinedFunction

toStr = UserDefinedFunction(lambda k: k, StringType())
CV_data = CV_data.withColumn('Churn', toStr(CV_data['Churn']))

#binary_map = {'Yes':1.0, 'No':0.0, 'True':1.0, 'False':0.0}
#toNum = UserDefinedFunction(lambda k: binary_map[k], DoubleType())
#CV_data = CV_data.withColumn('Churn', toNum(CV_data['Churn']))

In [11]:
CV_data.printSchema()

## Spark: ML Pipelines
https://spark.apache.org/docs/latest/ml-pipeline.html

##  <font color= #e38009> Transformer A: StringIndexer
  https://spark.apache.org/docs/latest/ml-features.html#stringindexer

<font font-family: "calibri" size=3.5>StringIndexer converts String values that are part of a look-up into categorical indices, which could be used by machine learning algorithms in ml library.

***Notice we provide the input column name and the output column name as parameters at the time of initialization of the StringIndexer.***

In [14]:
from pyspark.ml.feature import StringIndexer

# Index labels: using StringIndexer to encodes a string column of labels Churn ("True" , "False" strings NO Boolean) to a column of label indices indexedChurn

stringindexer = StringIndexer(inputCol='Churn',
                             outputCol='indexedChurn')

model=stringindexer.fit(CV_data)


dataframe_transformedA=model.transform(CV_data)

In [15]:
display(dataframe_transformedA)

##  <font color= #e38009> Transformer B: VectorAssembler

### ...after “feature engineering” … the feature engineering results are then combined using the VectorAssembler, before being passed to ML Estimator

###  For simplicity: first we drop all columns:
* categorical
* and numerical highly correlated

### This will be our list with predictors

In [19]:
predictors=('Number vmail messages',
 'Total day minutes',
 'Total day calls',
 'Total eve minutes',
 'Total eve calls',
 'Total night minutes',
 'Total night calls',
 'Total intl minutes',
 'Total intl calls',
 'Customer service calls')

#### Notice we provide to *VectorAssembler* the input = list of columns (MUST BE NUMERIC!) and the output column assembles all of them in a single column/vector

In [21]:
from pyspark.ml.feature import VectorAssembler

assembler=VectorAssembler(inputCols=predictors,outputCol='features')

dataframe_transformedB=assembler.transform(dataframe_transformedA).select('indexedChurn','features')


In [22]:
dataframe_transformedB.take(5)

##  <font color=#FF5733> Estimators

<font font-family: "calibri" size=3.5>
An Estimator abstracts the concept of a learning algorithm or any algorithm that fits or trains on data. 

Technically, an Estimator implements a method fit(), which accepts a DataFrame and produces a Model, which is a Transformer. <br><br>
***For example, a learning algorithm such as LogisticRegression is an Estimator, and calling fit() trains a LogisticRegressionModel, which is a Model and hence a Transformer.***

In [24]:
from pyspark.ml.classification import DecisionTreeClassifier

# Train a DecisionTree model
dTree_algorithm = DecisionTreeClassifier(maxDepth=2,
                                        labelCol='indexedChurn', featuresCol='features')

In [25]:
dTree_model=dTree_algorithm.fit(dataframe_transformedB)

In [26]:
print(dTree_model._call_java("toDebugString"))

##  <font color= #e38009> Transformers include:learned models: 

*** e.g.  take a DataFrame, read the column containing feature vectors, predict the label for each feature vector, and output a new DataFrame with predicted labels appended as a column***

In [28]:
predictions=dTree_model.transform(dataframe_transformedB)

In [29]:
predictions.printSchema()

In [30]:
display(predictions)

In [31]:
import pandas as pd

pd.DataFrame(predictions.take(5), columns=predictions.columns)

## <font color=#938882>Model Evaluation

### *** For evaluation we will use the training cvs file, that is Train Error***

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [34]:
evaluator=BinaryClassificationEvaluator(labelCol='indexedChurn',\
                                        rawPredictionCol='rawPrediction',\
                                       metricName='areaUnderROC')

In [35]:
accuracy=evaluator.evaluate(predictions)

In [36]:
accuracy

In [37]:
# Since dTree_model is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("dTree_model was fit using parameters: ")
print(dTree_model.extractParamMap())

In [38]:
dTree_model.extractParamMap().keys()

In [39]:
dTree_model.maxDepth

Out[ 95 ]: Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.')

In [40]:
# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {dTree_model.maxDepth: 1}
paramMap[dTree_model.maxDepth] = 7  # Specify 1 Param, overwriting the original maxIter.


# Now learn a new model using the paramMap parameters.
# paramMap overrides all parameters set earlier via dTree_model.set* methods.
dTree_model2=dTree_algorithm.fit(dataframe_transformedB, paramMap)

dTree_model2.extractParamMap()

Out[ 108 ]: 
{Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='featuresCol', doc='features column name'): 'features',
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini',
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='labelCol', doc='label column name'): 'indexedChurn',
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 7,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name'): 'rawPrediction',
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='seed', doc='random seed'): 6174923023070228847,
 Param(parent=u'DecisionTreeClassifier_47c98c8c31663453d066', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities'): 'probability'}

In [41]:
predictions2=dTree_model2.transform(dataframe_transformedB)

accuracy2=evaluator.evaluate(predictions2)
print(accuracy2)



0.711933623272

In [42]:
print(dTree_model2._call_java("toDebugString"))

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_47c98c8c31663453d066) of depth 7 with 139 nodes
 If (feature 1 <= 263.1)
 If (feature 9 <= 3.5)
 If (feature 1 <= 220.85000000000002)
 If (feature 7 <= 13.05)
 If (feature 8 <= 2.5)
 If (feature 3 <= 259.65)
 If (feature 4 <= 103.5)
 Predict: 0.0
 Else (feature 4 > 103.5)
 Predict: 0.0
 Else (feature 3 > 259.65)
 If (feature 6 <= 77.5)
 Predict: 1.0
 Else (feature 6 > 77.5)
 Predict: 0.0
 Else (feature 8 > 2.5)
 If (feature 3 <= 183.35000000000002)
 If (feature 2 <= 73.5)
 Predict: 0.0
 Else (feature 2 > 73.5)
 Predict: 0.0
 Else (feature 3 > 183.35000000000002)
 If (feature 8 <= 6.5)
 Predict: 0.0
 Else (feature 8 > 6.5)
 Predict: 0.0
 Else (feature 7 > 13.05)
 If (feature 0 <= 38.5)
 If (feature 8 <= 17.5)
 If (feature 9 <= 0.5)
 Predict: 0.0
 Else (feature 9 > 0.5)
 Predict: 0.0
 Else (feature 8 > 17.5)
 Predict: 1.0
 Else (feature 0 > 38.5)
 If (feature 6 <= 88.5)
 Predict: 1.0
 Else (feature 6 > 88.5)
 If (feature 1 <= 175.64999999999998)
 Predict: 0.0
 Else (feature 1 > 175.64999999999998)
 Predict: 1.0
 Else (feature 1 > 220.85000000000002)
 If (feature 3 <= 240.64999999999998)
 If (feature 7 <= 13.850000000000001)
 If (feature 8 <= 2.5)
 If (feature 6 <= 129.5)
 Predict: 0.0
 Else (feature 6 > 129.5)
 Predict: 1.0
 Else (feature 8 > 2.5)
 If (feature 5 <= 253.55)
 Predict: 0.0
 Else (feature 5 > 253.55)
 Predict: 0.0
 Else (feature 7 > 13.850000000000001)
 If (feature 3 <= 220.85000000000002)
 If (feature 6 <= 113.5)
 Predict: 0.0
 Else (feature 6 > 113.5)
 Predict: 1.0
 Else (feature 3 > 220.85000000000002)
 Predict: 1.0
 Else (feature 3 > 240.64999999999998)
 If (feature 0 <= 2.0)
 If (feature 5 <= 149.95)
 If (feature 8 <= 1.5)
 Predict: 1.0
 Else (feature 8 > 1.5)
 Predict: 0.0
 Else (feature 5 > 149.95)
 If (feature 7 <= 6.65)
 Predict: 0.0
 Else (feature 7 > 6.65)
 Predict: 1.0
 Else (feature 0 > 2.0)
 If (feature 7 <= 15.350000000000001)
 Predict: 0.0
 Else (feature 7 > 15.350000000000001)
 Predict: 1.0
 Else (feature 9 > 3.5)
 If (feature 1 <= 179.95)
 If (feature 3 <= 212.14999999999998)
 If (feature 5 <= 226.55)
 Predict: 1.0
 Else (feature 5 > 226.55)
 If (feature 3 <= 183.35000000000002)
 If (feature 1 <= 171.05)
 Predict: 1.0
 Else (feature 1 > 171.05)
 Predict: 0.0
 Else (feature 3 > 183.35000000000002)
 Predict: 0.0
 Else (feature 3 > 212.14999999999998)
 If (feature 1 <= 153.0)
 If (feature 8 <= 4.5)
 If (feature 5 <= 278.8)
 Predict: 1.0
 Else (feature 5 > 278.8)
 Predict: 0.0
 Else (feature 8 > 4.5)
 If (feature 1 <= 138.2)
 Predict: 1.0
 Else (feature 1 > 138.2)
 Predict: 0.0
 Else (feature 1 > 153.0)
 Predict: 0.0
 Else (feature 1 > 179.95)
 If (feature 3 <= 141.95)
 If (feature 4 <= 95.5)
 Predict: 1.0
 Else (feature 4 > 95.5)
 If (feature 5 <= 155.95)
 Predict: 1.0
 Else (feature 5 > 155.95)
 If (feature 6 <= 106.5)
 Predict: 0.0
 Else (feature 6 > 106.5)
 Predict: 1.0
 Else (feature 3 > 141.95)
 If (feature 6 <= 115.5)
 If (feature 9 <= 6.5)
 If (feature 6 <= 82.5)
 Predict: 0.0
 Else (feature 6 > 82.5)
 Predict: 0.0
 Else (feature 9 > 6.5)
 Predict: 1.0
 Else (feature 6 > 115.5)
 If (feature 2 <= 120.5)
 If (feature 8 <= 3.5)
 Predict: 1.0
 Else (feature 8 > 3.5)
 Predict: 0.0
 Else (feature 2 > 120.5)
 Predict: 1.0
 Else (feature 1 > 263.1)
 If (feature 0 <= 2.0)
 If (feature 3 <= 187.9)
 If (feature 1 <= 279.1)
 If (feature 5 <= 253.55)
 If (feature 2 <= 77.5)
 If (feature 2 <= 69.5)
 Predict: 0.0
 Else (feature 2 > 69.5)
 Predict: 1.0
 Else (feature 2 > 77.5)
 Predict: 0.0
 Else (feature 5 > 253.55)
 If (feature 3 <= 123.45)
 Predict: 0.0
 Else (feature 3 > 123.45)
 Predict: 1.0
 Else (feature 1 > 279.1)
 If (feature 8 <= 4.5)
 If (feature 7 <= 5.05)
 Predict: 0.0
 Else (feature 7 > 5.05)
 Predict: 1.0
 Else (feature 8 > 4.5)
 If (feature 7 <= 10.05)
 Predict: 0.0
 Else (feature 7 > 10.05)
 If (feature 4 <= 100.5)
 Predict: 1.0
 Else (feature 4 > 100.5)
 Predict: 0.0
 Else (feature 3 > 187.9)
 If (feature 5 <= 123.1)
 If (feature 6 <

##  Model selection via cross-validation

In this example we will use CrossValidator to select from a grid of parameters in the Tree model

In [44]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Search through decision tree's maxDepth parameter for best model
paramGrid = ParamGridBuilder().addGrid(dTree_algorithm.maxDepth, [2,3,4,5,6,7]).build()

In [45]:
# Set up 3-fold cross validation
crossval = CrossValidator(estimator=dTree_algorithm,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

In [46]:
Cross_res=crossval.fit(dataframe_transformedB)

In [47]:
print(Cross_res.bestModel)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_47c98c8c31663453d066) of depth 7 with 139 nodes

In [48]:
print(Cross_res.bestModel._call_java("toDebugString"))

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_47c98c8c31663453d066) of depth 7 with 139 nodes
 If (feature 1 <= 263.1)
 If (feature 9 <= 3.5)
 If (feature 1 <= 220.85000000000002)
 If (feature 7 <= 13.05)
 If (feature 8 <= 2.5)
 If (feature 3 <= 259.65)
 If (feature 4 <= 103.5)
 Predict: 0.0
 Else (feature 4 > 103.5)
 Predict: 0.0
 Else (feature 3 > 259.65)
 If (feature 6 <= 77.5)
 Predict: 1.0
 Else (feature 6 > 77.5)
 Predict: 0.0
 Else (feature 8 > 2.5)
 If (feature 3 <= 183.35000000000002)
 If (feature 2 <= 73.5)
 Predict: 0.0
 Else (feature 2 > 73.5)
 Predict: 0.0
 Else (feature 3 > 183.35000000000002)
 If (feature 8 <= 6.5)
 Predict: 0.0
 Else (feature 8 > 6.5)
 Predict: 0.0
 Else (feature 7 > 13.05)
 If (feature 0 <= 38.5)
 If (feature 8 <= 17.5)
 If (feature 9 <= 0.5)
 Predict: 0.0
 Else (feature 9 > 0.5)
 Predict: 0.0
 Else (feature 8 > 17.5)
 Predict: 1.0
 Else (feature 0 > 38.5)
 If (feature 6 <= 88.5)
 Predict: 1.0
 Else (feature 6 > 88.5)
 If (feature 1 <= 175.64999999999998)
 Predict: 0.0
 Else (feature 1 > 175.64999999999998)
 Predict: 1.0
 Else (feature 1 > 220.85000000000002)
 If (feature 3 <= 240.64999999999998)
 If (feature 7 <= 13.850000000000001)
 If (feature 8 <= 2.5)
 If (feature 6 <= 129.5)
 Predict: 0.0
 Else (feature 6 > 129.5)
 Predict: 1.0
 Else (feature 8 > 2.5)
 If (feature 5 <= 253.55)
 Predict: 0.0
 Else (feature 5 > 253.55)
 Predict: 0.0
 Else (feature 7 > 13.850000000000001)
 If (feature 3 <= 220.85000000000002)
 If (feature 6 <= 113.5)
 Predict: 0.0
 Else (feature 6 > 113.5)
 Predict: 1.0
 Else (feature 3 > 220.85000000000002)
 Predict: 1.0
 Else (feature 3 > 240.64999999999998)
 If (feature 0 <= 2.0)
 If (feature 5 <= 149.95)
 If (feature 8 <= 1.5)
 Predict: 1.0
 Else (feature 8 > 1.5)
 Predict: 0.0
 Else (feature 5 > 149.95)
 If (feature 7 <= 6.65)
 Predict: 0.0
 Else (feature 7 > 6.65)
 Predict: 1.0
 Else (feature 0 > 2.0)
 If (feature 7 <= 15.350000000000001)
 Predict: 0.0
 Else (feature 7 > 15.350000000000001)
 Predict: 1.0
 Else (feature 9 > 3.5)
 If (feature 1 <= 179.95)
 If (feature 3 <= 212.14999999999998)
 If (feature 5 <= 226.55)
 Predict: 1.0
 Else (feature 5 > 226.55)
 If (feature 3 <= 183.35000000000002)
 If (feature 1 <= 171.05)
 Predict: 1.0
 Else (feature 1 > 171.05)
 Predict: 0.0
 Else (feature 3 > 183.35000000000002)
 Predict: 0.0
 Else (feature 3 > 212.14999999999998)
 If (feature 1 <= 153.0)
 If (feature 8 <= 4.5)
 If (feature 5 <= 278.8)
 Predict: 1.0
 Else (feature 5 > 278.8)
 Predict: 0.0
 Else (feature 8 > 4.5)
 If (feature 1 <= 138.2)
 Predict: 1.0
 Else (feature 1 > 138.2)
 Predict: 0.0
 Else (feature 1 > 153.0)
 Predict: 0.0
 Else (feature 1 > 179.95)
 If (feature 3 <= 141.95)
 If (feature 4 <= 95.5)
 Predict: 1.0
 Else (feature 4 > 95.5)
 If (feature 5 <= 155.95)
 Predict: 1.0
 Else (feature 5 > 155.95)
 If (feature 6 <= 106.5)
 Predict: 0.0
 Else (feature 6 > 106.5)
 Predict: 1.0
 Else (feature 3 > 141.95)
 If (feature 6 <= 115.5)
 If (feature 9 <= 6.5)
 If (feature 6 <= 82.5)
 Predict: 0.0
 Else (feature 6 > 82.5)
 Predict: 0.0
 Else (feature 9 > 6.5)
 Predict: 1.0
 Else (feature 6 > 115.5)
 If (feature 2 <= 120.5)
 If (feature 8 <= 3.5)
 Predict: 1.0
 Else (feature 8 > 3.5)
 Predict: 0.0
 Else (feature 2 > 120.5)
 Predict: 1.0
 Else (feature 1 > 263.1)
 If (feature 0 <= 2.0)
 If (feature 3 <= 187.9)
 If (feature 1 <= 279.1)
 If (feature 5 <= 253.55)
 If (feature 2 <= 77.5)
 If (feature 2 <= 69.5)
 Predict: 0.0
 Else (feature 2 > 69.5)
 Predict: 1.0
 Else (feature 2 > 77.5)
 Predict: 0.0
 Else (feature 5 > 253.55)
 If (feature 3 <= 123.45)
 Predict: 0.0
 Else (feature 3 > 123.45)
 Predict: 1.0
 Else (feature 1 > 279.1)
 If (feature 8 <= 4.5)
 If (feature 7 <= 5.05)
 Predict: 0.0
 Else (feature 7 > 5.05)
 Predict: 1.0
 Else (feature 8 > 4.5)
 If (feature 7 <= 10.05)
 Predict: 0.0
 Else (feature 7 > 10.05)
 If (feature 4 <= 100.5)
 Predict: 1.0
 Else (feature 4 > 100.5)
 Predict: 0.0
 Else (feature 3 > 187.9)
 If (feature 5 <= 123.1)
 If (feature 6 <

In [49]:
# Fetch the best model for make predictions with it:
Best_tree_model = Cross_res.bestModel
print(Best_tree_model)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_47c98c8c31663453d066) of depth 7 with 139 nodes

In [50]:
predictions_CV=Best_tree_model.transform(dataframe_transformedB)

In [51]:
pd.DataFrame(predictions_CV.take(5), columns=predictions.columns)

Out[ 118 ]: 
 indexedChurn features \
0 0.0 [25.0, 265.1, 110.0, 197.4, 99.0, 244.7, 91.0,... 
1 0.0 [26.0, 161.6, 123.0, 195.5, 103.0, 254.4, 103.... 
2 0.0 [0.0, 243.4, 114.0, 121.2, 110.0, 162.6, 104.0... 
3 0.0 [0.0, 299.4, 71.0, 61.9, 88.0, 196.9, 89.0, 6.... 
4 0.0 [0.0, 166.7, 113.0, 148.3, 122.0, 186.9, 121.0... 

 rawPrediction probability prediction 
0 [29.0, 0.0] [1.0, 0.0] 0.0 
1 [190.0, 18.0] [0.913461538462, 0.0865384615385] 0.0 
2 [159.0, 4.0] [0.975460122699, 0.0245398773006] 0.0 
3 [5.0, 0.0] [1.0, 0.0] 0.0 
4 [461.0, 4.0] [0.991397849462, 0.00860215053763] 0.0

In [52]:
accuracy_CV=evaluator.evaluate(predictions_CV)

print(accuracy_CV)

0.711933623272

## Now let's create a PIPELINE! see MSTC_Pipeline_PySpark_2.ipynb